Projet kaggle : Tweet Sentiment Extraction

Emma BEUZE, Nacim BOUIA, Antoine FRANCESCHI, Ayoub RHIM, Guillaume RUSSIAS

In [ ]:
#???
import jax
import jax.numpy as jnp
import gzip
import pickle

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
#besoin d'une version > à 1.2.0
pd.__version__

'1.2.4'

Il y a 4 types de colonnes :


*   textID - unique ID for each piece of text 
*   text - the text of the tweet 
*   sentiment - the general sentiment of the tweet        => il y a 3 sentiments : positive, neutral, negative
*   selected_text - [train only] the text that supports the tweet's sentiment 


**OBJECTIF :  examiner le sentiment donné pour un tweet donné et déterminer le mot ou la phrase qui le soutient le mieux.**






In [4]:
os.chdir( os.getcwd() + "/donnees" )

Page qui résume comment utiliser les pandas : https://ledatascientist.com/manipulez-vos-donnees-avec-pandas/

In [5]:
# Load the dataset
train_set = pd.read_csv(r"train.csv")
test_set = pd.read_csv(r"test.csv")

#print(train_set.shape)    => (27481, 4)  (textID, text, selected_text, sentiment )
#print(test_set.shape)     => (3534, 3)   (textID, text, sentiment)

train_data_text = train_set["text"]
train_data_sentiment = train_set["sentiment"]

N_train = train_data_text.shape[0]

train_data = [(train_data_text[k],train_data_sentiment[k]) for k in range(N_train)]
train_labels = train_set["selected_text"]


# check data makes sense

print("Exemple 1 :",train_data[1], "extrait choisi :", train_labels[1])
print("Exemple 2 :",train_data[59], "extrait choisi :", train_labels[59])
print("Exemple 3 :",train_data[750], "extrait choisi :", train_labels[750])

Exemple 1 : (' Sooo SAD I will miss you here in San Diego!!!', 'negative') extrait choisi : Sooo SAD
Exemple 2 : (' what fun are you speaking of?', 'neutral') extrait choisi : what fun are you speaking of?
Exemple 3 : (' Thanks! Welcome back!', 'positive') extrait choisi : Thanks! Welcome back!


Dans le concours, la métrique à utiliser est le score de Jaccard : </br>
https://en.wikipedia.org/wiki/Jaccard_index </br>
https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50.


In [6]:
class RetardedClassifier():
    def __init__(self, n = 2000):
        self._train_frame = None
        self._train_extracts = None
        self._depth = n
    
    def fit(self, train_set):
        self._train_frame = train_set.loc[1:self._depth,("textID","text","sentiment")]
        self._train_extracts = train_set.loc[1:self._depth,("textID","selected_text")]
    
    def __call__(self, new_frame):
        #TODO :)
        return

In [7]:
R = RetardedClassifier()
R.fit(train_set)
R._train_frame
test_set.head()


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [9]:
aux = R._train_frame.merge(test_set, how='cross')
aux.head()

,textID_x,text_x,sentiment_x,textID_y,text_y,sentiment_y
0,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,01082688c6,happy bday!,positive
4,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [10]:
def jaccard(str1, str2): 
    try:
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    except:
        print(str1, str2)

In [11]:
str1 = "je m'appelle Emma blablabla"
str2 = "je m'appelle Emma blablabla"
jaccard(str1,str2)

1.0

In [12]:
def row_jac(row):
    return jaccard(row.text_x, row.text_y)

aux2 = aux.loc[:,["text_x","text_y"]]
aux2["score"] = aux.apply(row_jac, axis = 1) 

nan Last session of the day  http://twitpic.com/67ezh
nan  Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China:  (SH)  (BJ).
nan Recession hit Veronique Branquinho, she has to quit her company, such a shame!
nan  happy bday!
nan  http://twitpic.com/4w75p - I like it!!
nan  that`s great!! weee!! visitors!
nan I THINK EVERYONE HATES ME ON HERE   lol
nan  soooooo wish i could, but im in school and myspace is completely blocked
nan  and within a short time of the last clue all of them
nan  What did you get?  My day is alright.. haven`t done anything yet. leaving soon to my stepsister though!
nan My bike was put on hold...should have known that.... argh total bummer
nan  I checked.  We didn`t win
nan  .. and you`re on twitter! Did the tavern bore you that much?
nan I`m in VA for the weekend, my youngest son turns 2 tomorrow......it makes me kinda sad, he is getting so big, check out my twipics
nan Its coming out the socket  I feel like my phones hole is 

nan   sad face.
nan Aaaaahhhh.... Friday!!!! but.......     Funeral at 5
nan Already misses 
nan _laura_LP ok thanks for the help! I hope they respond to me sooner or later, then! thanks!
nan   out of commish for a few days in ATL with fam.  Just saw the updates to the .net website... Looks AWESOME!
nan  awww that sucks  also, when i finish uni, you should totally have a mini housewarming. yes, work it around my schedule, LOL.
nan at famous dave`s with my mom. so hungry.
nan  I would vote for you Miley but I live in Ireland  You`ll be fine  Good luck! Slï¿½n!(bye in irish ) Sorcha xxxx
nan home for the nite, sleeping on the pull out bed with Maggie. wishing it was _durden  I got to find a apt
nan Oh..Summer begins..yay..  The fighting, Crying and stress has started with the kids..It`s only been 3 hrs since they got out.
nan I feel so deflated. No more doggy.
nan  sad about young love coming to an end  They were sooo cute together!
nan  Omg really??  I sowwy! I`ll do it again! :-p I was

nan Happy Mother`s Day mom
nan  http://twitpic.com/4vw9a - I`m not gonna buy one...nor two... But three! I will be the ultimate slaughter machine!
nan #asylm yay registered but lost all my friends
nan church bellss are ringin  got to go. adieu,(god bless in german)
nan Still feel like craaaap and like everyone on earth needs to know this lol... Have fun birdies
nan  watching dor on utv movies again great performance
nan did U watch the spelling bee contest ? winner: Kavya Shivshankar ! Impressive performance - i had not even heared most of the words b4
nan  probably not in this weather
nan some relaxation. so much things on my list... but really not that much
nan ....got your message!!! You are such a twitter freak!
nan  no wonder you hate driving and country roads and stuff
nan heading out to (a long)band practice. last one before our first show on Saturday night. Hope it`s a good one  http://digg.com/u12MIW
nan says HAPPY MOTHER`S DAY TO ALL THE MOMS OUT THERE!  http://plurk.com/p/st

nan Is my picture showing?
nan about to take my mum out for jewellery shopping and a coffee for mothers day
nan so who is in for bring at HK lounge tomorrow ? $12 all you can drink till 5pm... then kareoke. I think it will be a nice Sunday!
nan  oh ****! I completely forgot about Thirsty Thursday! oh. my. gaga. this sucks
nan I wanna feel my chin
nan  i do but is just a haul vid/photo slideshow only. I only do it for fun  http://www.youtube.com/eyezaddiction
nan feeling so great!
nan AHHHHHHHH!I can`t get miley to notice me......
nan Coffee brewing, on this #musicmonday morning i`m #listening to the fray... Here`s to a good week
nan I Wanna be @ JET in the Mirage...     Couldn`t get to Hawaii either   *****Someday***** Much Love To THE_WOCKEEZ and SuperCr3w
nan Just finished yet another amazing book by Susan Elizabeth Philips!!!  God I love her books so much!!! Which one to read next??? hmmm...
nan _O_C I`m seriously screwed  I haven`t studied at all!!!! :`(
nan well being a retired dr

nan  finished listening to Last Days of Krypton`s audio book! Loved it...so sad though...
nan  A rare treat b/c we`re rarely ap and at `em that early!  But we may make opening bell at the PDX Farmers Mkt a new goal.
nan Dying to get my hands on the Diagnosis Murder DVD boxset but those pesky kids at Amazon still won`t deliver to Zimbabwe
nan Thank you all new, first time Japa Dog customers! Twitter spreads the word! We like new customers
nan  - I read that as follows: 'I`m a what?! What`d I do? Yes, she is rather simple. I miss you dicks too...   '
nan New chic just texted me saying 'Thanks for the e-mail' but didn`t answer to my question, might need to look for newer chic now
nan _Fur_Laxis It makes me feel physically sick. I read Let The Right One In and was only horrified by the cat violence.  Just horrible
nan  Good Grief! I can`t say much, I was driving home from Martin through IKE last fall. Be interesting to hear what he says!
nan storming outside
nan  as a proper journo I would

nan hmm icecream for breakfast  #springtime
nan Wishing I could have seen my nephew graduate from kindergarten!
nan  i (L u so much  See u at school todaay!
nan Britain`s got Torture - Edelcries...
nan   its suzi
nan  It`s amazing the DVDs I`ll put on my Netflix list just based on the trash potential inherent in their titles
nan _august84 - requoting the gyaan - the more you live life, the more you fail to notice how you live it - quote By muthu...
nan perkinz..  idk wat after dat tho. maybe sum ****. lol
nan   You`re silly.
nan It`s 1:15 AM, and I hear a helicopter flying over my house. That`s kind of weird. In other news, my headache is back. BOOOOO.
nan  Rofl u did?  Hahaha! Lemme tell Salman Munir =P =P
nan  YEA!! Way to go! Super excited for a great Midwest resource!!! Midwest pride!
nan Hi .. I have the net YAYY.. Im here for a short time though
nan Loving the sunshine! Wish poor Richie would feel better
nan and we`re in
nan  heyy i have a sis of 1 year with 5 months and still sh

In [18]:
help(pd.DataFrame.groupby)

Help on function groupby in module pandas.core.frame:

groupby(self, by=None, axis=0, level=None, as_index: 'bool' = True, sort: 'bool' = True, group_keys: 'bool' = True, squeeze: 'bool' = <object object at 0x7fa5a9c7bde0>, observed: 'bool' = False, dropna: 'bool' = True) -> 'DataFrameGroupBy'
    Group DataFrame using a mapper or by a Series of columns.
    
    A groupby operation involves some combination of splitting the
    object, applying a function, and combining the results. This can be
    used to group large amounts of data and compute operations on these
    groups.
    
    Parameters
    ----------
    by : mapping, function, label, or list of labels
        Used to determine the groups for the groupby.
        If ``by`` is a function, it's called on each value of the object's
        index. If a dict or Series is passed, the Series or dict VALUES
        will be used to determine the groups (the Series' values are first
        aligned; see ``.align()`` method). If an nd

In [18]:
aux2

,text_x,text_y,score
0,Sooo SAD I will miss you here in San Diego!!!,Last session of the day http://twitpic.com/67ezh,0.000000
1,Sooo SAD I will miss you here in San Diego!!!,Shanghai is also really exciting (precisely -...,0.041667
2,Sooo SAD I will miss you here in San Diego!!!,"Recession hit Veronique Branquinho, she has to...",0.000000
3,Sooo SAD I will miss you here in San Diego!!!,happy bday!,0.000000
4,Sooo SAD I will miss you here in San Diego!!!,http://twitpic.com/4w75p - I like it!!,0.071429
...,...,...,...
7067995,going to eat rice and fried chicken from seder...,"its at 3 am, im very tired but i can`t sleep ...",0.000000
7067996,going to eat rice and fried chicken from seder...,All alone in this old house again. Thanks for...,0.055556
7067997,going to eat rice and fried chicken from seder...,I know what you mean. My little dog is sinkin...,0.033333
7067998,going to eat rice and fried chicken from seder...,_sutra what is your next youtube video gonna b...,0.000000


In [13]:
aux3 = aux2.sort_values('score', ascending=False).drop_duplicates(['text_x'])

In [22]:
aux3[aux3.score == 0]

,text_x,text_y,score
4659716,ShoesShoesShoes.YayYayYay.lol.IWouldPostATwitP...,arrr bummer who do you want to win?,0.0
5376214,Byebye,"i get so sucked in to true life, and that stev...",0.0
1410516,qood morninq,why aren`t you showing up as a #spymaster in ...,0.0
1159952,?sucks!?..,I would vote for you Miley but I live in Irel...,0.0
1378664,BRAINFREEZE,happy mommah`s day to your moms http://plurk....,0.0
2023363,.....headache,eeek!! Your coming!!!! Im soo excited to s...,0.0
2035201,mozart`s requiem!,you writing fanfic? me read!,0.0
145115,MAYDAY?!,it`s very pleasant. i sooo rarely just have ...,0.0
237233,Chilliin,Going to bed after the CAVS game Don`t judge ...,0.0
788876,_nesmith,Aaaaahhhh.... Friday!!!! but....... Funera...,0.0
